In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import csv
import time

In [75]:
# Load the Dataset
text = ""
with open("/kaggle/input/poemds/poems-100.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        text += " ".join(row) + " "                          
# Combine All Lines into a Single Text

In [76]:
# Tokenize the Text into Words
tokens = text.split()

In [77]:
# Create a Dictionary to Map Words to Indices
word_to_idx = {}
idx_to_word = {}
vocab_size = 0

for word in tokens:
    if word not in word_to_idx:
        word_to_idx[word] = vocab_size
        idx_to_word[vocab_size] = word
        vocab_size += 1

In [78]:
print(f"Vocabulary Size: {vocab_size}")

Vocabulary Size: 7460


In [79]:
# Convert Tokens to Indices
token_indices = [word_to_idx[word] for word in tokens]

In [80]:
# Create Sequences and Targets
seq_length = 10
sequences = []
targets = []

for i in range(len(token_indices) - seq_length):
    seq = token_indices[i:i + seq_length]
    target = token_indices[i + seq_length]
    sequences.append(seq)
    targets.append(target)

In [81]:
# Convert to PyTorch Tensors
sequences = torch.tensor(sequences, dtype = torch.long)
targets = torch.tensor(targets, dtype = torch.long)

In [82]:
# Define One-Hot Encoding for RNN Model
class OneHotRNN(nn.Module):
    def __init__(self, vocab_size, hidden_dim, output_dim):
        super(OneHotRNN, self).__init__()
        self.rnn = nn.RNN(vocab_size, hidden_dim, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        output, _ = self.rnn(x)
        out = self.fc(output[:, -1, :])
        return out

In [83]:
# Define LSTM Model with Embedding Layer
class PoemLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(PoemLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        out = self.fc(output[:, -1, :])
        return out

In [84]:
# Hyperparameters
embed_dim = 100
hidden_dim = 128
output_dim = vocab_size

In [85]:
# Initialize Models
onehot_model = OneHotRNN(vocab_size, hidden_dim, output_dim)
embedding_model = PoemLSTM(vocab_size, embed_dim, hidden_dim, output_dim)

In [86]:
criterion = nn.CrossEntropyLoss()
onehot_optimizer = optim.Adam(onehot_model.parameters(), lr = 0.001)
embedding_optimizer = optim.Adam(embedding_model.parameters(), lr = 0.001)

In [87]:
# Loss Tracking
onehot_losses, embedding_losses = [], []

In [89]:
# Training Function with Tracking
def train_model(model, optimizer, name):
    start_time = time.time()
    for epoch in range(100):
        total_loss = 0
        for i in range(0, len(sequences), 32):
            batch_seq = sequences[i:i + 32]
            batch_target = targets[i:i + 32]

            # One-Hot Encoding for OneHotRNN
            if name == "OneHotRNN":
                batch_seq = F.one_hot(batch_seq, num_classes = vocab_size).float()

            # Forward Pass
            outputs = model(batch_seq)
            loss = criterion(outputs, batch_target)

            # Backward Pass and Optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / (len(sequences) // 32)
        if name == "OneHotRNN":
            onehot_losses.append(avg_loss)
        else:
            embedding_losses.append(avg_loss)

        print(f"{name} Epoch [{epoch+1}/100], Avg Loss: {avg_loss:.4f}")
    print(f"{name} Training Time: {time.time() - start_time:.2f}s\n")

In [90]:
# Poem Generation Function
def generate_poem(model, seed_text, num_words = 50, model_type = "EmbeddingLSTM"):
    model.eval()
    words = seed_text.split()
    with torch.no_grad():
        for _ in range(num_words):
            seq = [word_to_idx.get(word, 0) for word in words[-seq_length:]]
            seq = torch.tensor(seq, dtype = torch.long).unsqueeze(0)

            if model_type == "OneHotRNN":
                seq = F.one_hot(seq, num_classes = vocab_size).float()

            output = model(seq)
            probabilities = F.softmax(output, dim = 1)
            predicted_idx = torch.multinomial(probabilities, 1).item()

            words.append(idx_to_word[predicted_idx])

    return " ".join(words)

In [91]:
# Train Models
train_model(onehot_model, onehot_optimizer, "OneHotRNN")
train_model(embedding_model, embedding_optimizer, "EmbeddingLSTM")

OneHotRNN Epoch [1/100], Avg Loss: 7.5849
OneHotRNN Epoch [2/100], Avg Loss: 6.7192
OneHotRNN Epoch [3/100], Avg Loss: 6.3652
OneHotRNN Epoch [4/100], Avg Loss: 6.1406
OneHotRNN Epoch [5/100], Avg Loss: 5.9428
OneHotRNN Epoch [6/100], Avg Loss: 5.8024
OneHotRNN Epoch [7/100], Avg Loss: 5.5409
OneHotRNN Epoch [8/100], Avg Loss: 5.3024
OneHotRNN Epoch [9/100], Avg Loss: 5.1020
OneHotRNN Epoch [10/100], Avg Loss: 4.9088
OneHotRNN Epoch [11/100], Avg Loss: 4.6854
OneHotRNN Epoch [12/100], Avg Loss: 4.4943
OneHotRNN Epoch [13/100], Avg Loss: 4.2669
OneHotRNN Epoch [14/100], Avg Loss: 4.0271
OneHotRNN Epoch [15/100], Avg Loss: 3.8095
OneHotRNN Epoch [16/100], Avg Loss: 3.6117
OneHotRNN Epoch [17/100], Avg Loss: 3.4663
OneHotRNN Epoch [18/100], Avg Loss: 3.2921
OneHotRNN Epoch [19/100], Avg Loss: 3.0858
OneHotRNN Epoch [20/100], Avg Loss: 2.8735
OneHotRNN Epoch [21/100], Avg Loss: 2.7058
OneHotRNN Epoch [22/100], Avg Loss: 2.5675
OneHotRNN Epoch [23/100], Avg Loss: 2.4161
OneHotRNN Epoch [24/

In [92]:
# Generate Poems
seed_text = "I wandered lonely as a"
print("\nGenerated Poem (OneHotRNN):", generate_poem(onehot_model, seed_text, model_type = "OneHotRNN"))
print("\nGenerated Poem (EmbeddingLSTM):", generate_poem(embedding_model, seed_text, model_type = "EmbeddingLSTM"))


Generated Poem (OneHotRNN): I wandered lonely as a woman in by boots to land, Look at the It is a melancholy show better hundred before, But I would send them. let me a melancholy said of the little steady will never laugh and ever so vapor to the brook heart which poured Annabel life or an reach A

Generated Poem (EmbeddingLSTM): I wandered lonely as a child that it with a love of love made there can shut while I know Mine they have you have I guess it is just as lucky to die, and I know it. I pass death with the dying and birth with the new-wash'd babe, and am not contain'd between
